In [148]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from tensorflow import keras
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.activations import relu
import joblib
import plotly.io as pio

pio.renderers.default = 'jupyterlab'

In [149]:
new_data1 = joblib.load('new_data1.joblib')
new_data1.head(3)

,BuildingType,PrimaryPropertyType,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions
0,NonResidential,Hotel,downtown,47.61220,-122.33799,1927,1.0,12,88434,7226362.5,2003882.0,3946027.0,1276453.0,249.98
1,NonResidential,Hotel,downtown,47.61317,-122.33393,1996,1.0,11,103566,8387933.0,0.0,3242851.0,5145082.0,295.86
2,NonResidential,Hotel,downtown,47.61393,-122.33810,1969,1.0,41,956110,72587024.0,21566554.0,49526664.0,1493800.0,2089.28


In [150]:
new_data1['SiteEnergyUse(kBtu)'].describe()

count    1.666000e+03
mean     8.437933e+06
std      3.024380e+07
min      0.000000e+00
25%      1.229291e+06
50%      2.554947e+06
75%      6.913348e+06
max      8.739237e+08
Name: SiteEnergyUse(kBtu), dtype: float64

### encoder 'BuildingType', 'PrimaryPropertyType', 'Neighborhood',

In [151]:
def get_ohe(df, column_name):
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(df[[column_name]])
    temp_df = pd.DataFrame(data=ohe.transform(df[[column_name]]), columns=ohe.get_feature_names_out())
    df.drop(columns=[column_name], axis=1, inplace=True)
    df = pd.concat([df.reset_index(drop=True), temp_df], axis=1)
    return df

new_data1 = get_ohe(new_data1, 'BuildingType')
new_data1 = get_ohe(new_data1, 'PrimaryPropertyType')
new_data1 = get_ohe(new_data1, 'Neighborhood')


In [152]:
new_data1.head(2)

,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),...,Neighborhood_downtown,Neighborhood_east,Neighborhood_greater duwamish,Neighborhood_lake union,Neighborhood_magnolia / queen anne,Neighborhood_north,Neighborhood_northeast,Neighborhood_northwest,Neighborhood_southeast,Neighborhood_southwest
0,47.61220,-122.33799,1927,1.0,12,88434,7226362.5,2003882.0,3946027.0,1276453.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,47.61317,-122.33393,1996,1.0,11,103566,8387933.0,0.0,3242851.0,5145082.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
new_data1.columns

Index(['Latitude', 'Longitude', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFATotal', 'SiteEnergyUse(kBtu)',
       'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
       'TotalGHGEmissions', 'BuildingType_Campus',
       'BuildingType_NonResidential', 'BuildingType_Nonresidential COS',
       'BuildingType_Nonresidential WA', 'BuildingType_SPS-District K-12',
       'PrimaryPropertyType_Distribution Center',
       'PrimaryPropertyType_Hospital', 'PrimaryPropertyType_Hotel',
       'PrimaryPropertyType_K-12 School', 'PrimaryPropertyType_Laboratory',
       'PrimaryPropertyType_Large Office',
       'PrimaryPropertyType_Low-Rise Multifamily',
       'PrimaryPropertyType_Medical Office',
       'PrimaryPropertyType_Mixed Use Property', 'PrimaryPropertyType_Office',
       'PrimaryPropertyType_Other',
       'PrimaryPropertyType_Refrigerated Warehouse',
       'PrimaryPropertyType_Residence Hall', 'PrimaryPropertyType_Restaurant',
       'PrimaryProper

In [154]:
new_data1.shape

(1666, 51)

## Site Energy Use(kBtu)

In [155]:
#Je sélectionne les colonne pour la mettre dans la variable Y
y = new_data1[['SiteEnergyUse(kBtu)']]

#Je sélectionne toutes les colonnes sauf la dernière (qui est ma cible) pour les mettre dans la variable X
X = new_data1.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions', 'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)'], axis=1)

In [156]:

std_scale = preprocessing.StandardScaler().fit(X)
X_std = std_scale.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.25)

In [157]:
print("X:", X.shape, "Y:", y.shape)

in_dim = X.shape[1]
out_dim = y.shape[1]

X: (1666, 46) Y: (1666, 1)


In [158]:
model = Sequential()

model.add(Dense(2000, input_dim=in_dim, activation="relu"))
model.add(Dense(600, activation="relu"))
model.add(Dense(400, activation="relu"))
model.add(Dense(out_dim))


model.compile(loss="mae", optimizer= 'adam')
 

# dans mon cas, loss='mean_squared_error', (activation='relu' for hidden layers), (activation=linear' for output layers because this is regression )

In [159]:
model.summary()

Model: "sequential_105"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_319 (Dense)           (None, 2000)              94000     
                                                                 
 dense_320 (Dense)           (None, 600)               1200600   
                                                                 
 dense_321 (Dense)           (None, 400)               240400    
                                                                 
 dense_322 (Dense)           (None, 1)                 401       
                                                                 
Total params: 1,535,401
Trainable params: 1,535,401
Non-trainable params: 0
_________________________________________________________________


In [160]:
history = model.fit(X_train, 
                    y_train, 
                    epochs = 50,
                    batch_size = 4,
                    verbose = 1)

Epoch 1/50
313/313 [==============================] - 11s 21ms/step - loss: 7676870.0000
Epoch 2/50
313/313 [==============================] - 5s 16ms/step - loss: 6168893.5000
Epoch 3/50
313/313 [==============================] - 5s 17ms/step - loss: 5750513.0000
Epoch 4/50
313/313 [==============================] - 7s 21ms/step - loss: 5450444.0000
Epoch 5/50
313/313 [==============================] - 6s 18ms/step - loss: 5236955.5000
Epoch 6/50
313/313 [==============================] - 5s 17ms/step - loss: 5059956.0000
Epoch 7/50
313/313 [==============================] - 5s 15ms/step - loss: 4849829.0000
Epoch 8/50
313/313 [==============================] - 5s 15ms/step - loss: 4720249.5000
Epoch 9/50
313/313 [==============================] - 4s 12ms/step - loss: 4572624.5000
Epoch 10/50
313/313 [==============================] - 4s 11ms/step - loss: 4451479.5000
Epoch 11/50
313/313 [==============================] - 4s 13ms/step - loss: 4313127.5000
Epoch 12/50
313/313 [========

In [161]:
y_pred = model.predict(X_test)
result = model.evaluate(X_test, y_test, verbose=1)

result


14/14 [==============================] - 1s 4ms/step - loss: 4159251.2500


4159251.25

In [162]:
print("MAE : ", mean_absolute_error(y_test, y_pred))
print("R2 : ", r2_score(y_test, y_pred))
print("MSE : ", mean_squared_error(y_test, y_pred))
print("MAPE : ", mean_absolute_percentage_error(y_test, y_pred))

MAE :  4159250.964718465
R2 :  0.6633464345841464
MSE :  171115018560758.56
MAPE :  1.0513110630141326e+20


## GridSearchCV avec MLPRegressor

In [163]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
score = 'neg_mean_absolute_error'
params = {'activation': ['relu'],
          'hidden_layer_sizes': [(1000,600,400), (800,300,100), (600,50,30)],
          'solver': ['adam'],
          'max_iter' : [50, 80],
          'batch_size' : [2,4]
         }

mlp_regressor_grid = GridSearchCV(MLPRegressor(random_state=123), param_grid=params, n_jobs=-1, cv=5, verbose=5, scoring = score)
mlp_regressor_grid.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_mult

[CV 2/5] END activation=relu, batch_size=2, hidden_layer_sizes=(1000, 600, 400), max_iter=50, solver=adam;, score=-8890560.137 total time=27.7min


/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/5] END activation=relu, batch_size=2, hidden_layer_sizes=(1000, 600, 400), max_iter=50, solver=adam;, score=-4835699.143 total time=28.2min


...

[CV 4/5] END activation=relu, batch_size=4, hidden_layer_sizes=(600, 50, 30), max_iter=80, solver=adam;, score=-5460709.332 total time=  58.4s


/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 5/5] END activation=relu, batch_size=4, hidden_layer_sizes=(600, 50, 30), max_iter=80, solver=adam;, score=-6848595.626 total time=  45.9s


/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.



GridSearchCV(cv=5, estimator=MLPRegressor(random_state=123), n_jobs=-1,
             param_grid={'activation': ['relu'], 'batch_size': [2, 4],
                         'hidden_layer_sizes': [(1000, 600, 400),
                                                (800, 300, 100),
                                                (600, 50, 30)],
                         'max_iter': [50, 80], 'solver': ['adam']},
             scoring='neg_mean_absolute_error', verbose=5)

In [164]:

print('### Printing Best parameters ###')

mlp_regressor_grid.best_params_

### Printing Best parameters ###


{'activation': 'relu',
 'batch_size': 2,
 'hidden_layer_sizes': (600, 50, 30),
 'max_iter': 80,
 'solver': 'adam'}

In [165]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,r2_score

best_grid = mlp_regressor_grid.best_estimator_
y_test_pred_mlpcv = best_grid.predict(X_test)

mae_mlpcv = mean_absolute_error(y_test, y_test_pred_mlpcv)
mse_mlpcv = mean_squared_error(y_test, y_test_pred_mlpcv)
mape_mlpcv = mean_absolute_percentage_error(y_test, y_test_pred_mlpcv)
r2_mlpcv = r2_score(y_test,y_test_pred_mlpcv)

In [166]:
best_grid

MLPRegressor(batch_size=2, hidden_layer_sizes=(600, 50, 30), max_iter=80,
             random_state=123)

In [167]:
print("MAE_CV", mae_mlpcv)
print("r2_CV", r2_mlpcv)
print("MSE_CV", mse_mlpcv)
print("MAPE_CV", mape_mlpcv)


MAE_CV 5288814.791465753
r2_CV 0.6535169249440796
MSE_CV 176111183453370.12
MAPE_CV 1.266078634355688e+20


## Total GHG Emissions

In [168]:
#Je sélectionne les colonne pour la mettre dans la variable Y
y2 = new_data1[['TotalGHGEmissions']]

#Je sélectionne toutes les colonnes sauf la dernière (qui est ma cible) pour les mettre dans la variable X
X2 = new_data1.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions', 'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)'], axis=1)

In [169]:
std_scale = preprocessing.StandardScaler().fit(X2)
X_std2 = std_scale.transform(X2)

X_train, X_test, y_train, y_test = train_test_split(X_std2, y2, test_size=0.25)

In [170]:
print("X:", X2.shape, "Y:", y2.shape)

in_dim = X.shape[1]
out_dim = y.shape[1]

X: (1666, 46) Y: (1666, 1)


In [171]:
model = Sequential()

model.add(Dense(80, input_dim=in_dim, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(out_dim))
model.compile(loss="mae", optimizer= 'adam')

model.summary() 

Model: "sequential_106"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_323 (Dense)           (None, 80)                3760      
                                                                 
 dense_324 (Dense)           (None, 16)                1296      
                                                                 
 dense_325 (Dense)           (None, 1)                 17        
                                                                 
Total params: 5,073
Trainable params: 5,073
Non-trainable params: 0
_________________________________________________________________


In [172]:
history = model.fit(X_train, 
                    y_train, 
                    epochs = 100,
                    batch_size = 4,
                    verbose = 1)

Epoch 1/100
313/313 [==============================] - 3s 3ms/step - loss: 163.9613
Epoch 2/100
313/313 [==============================] - 1s 2ms/step - loss: 141.4058
Epoch 3/100
313/313 [==============================] - 1s 2ms/step - loss: 133.9823
Epoch 4/100
313/313 [==============================] - 0s 2ms/step - loss: 130.2024
Epoch 5/100
313/313 [==============================] - 0s 2ms/step - loss: 127.4288
Epoch 6/100
313/313 [==============================] - 0s 1ms/step - loss: 125.0608
Epoch 7/100
313/313 [==============================] - 0s 1ms/step - loss: 123.0861
Epoch 8/100
313/313 [==============================] - 0s 1ms/step - loss: 121.0210
Epoch 9/100
313/313 [==============================] - 1s 2ms/step - loss: 118.5568
Epoch 10/100
313/313 [==============================] - 0s 1ms/step - loss: 116.3748
Epoch 11/100
313/313 [==============================] - 0s 1ms/step - loss: 114.2980
Epoch 12/100
313/313 [==============================] - 1s 2ms/step - loss

In [173]:
y_pred2 = model.predict(X_test)
result2 = model.evaluate(X_test, y_test, verbose=1)

result2

14/14 [==============================] - 0s 3ms/step - loss: 145.1495


145.14950561523438

In [174]:
print("MAE : ", mean_absolute_error(y_test,y_pred2))
print("r2 : ", r2_score(y_test,y_pred2))
print("MSE : ", mean_squared_error(y_test,y_pred2))
print("MAPE : ", mean_absolute_percentage_error(y_test,y_pred2))

MAE :  145.1494979496454
r2 :  0.2493895177115536
MSE :  741997.4104063968
MAPE :  1684022358221961.5


In [175]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
score = 'neg_mean_absolute_error'
params = {'activation': ['relu'],
          'hidden_layer_sizes': [(80,16), (100,50), (80,10)],
          'solver': ['adam', 'lbfgs'],
          'max_iter' : [100, 80],
          'batch_size' : [2,4]
         }

mlp_regressor_grid2 = GridSearchCV(MLPRegressor(random_state=123), param_grid=params, n_jobs=-1, cv=5, verbose=5, scoring = score)
mlp_regressor_grid2.fit(X_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_mult

[CV 2/5] END activation=relu, batch_size=2, hidden_layer_sizes=(80, 16), max_iter=100, solver=adam;, score=-271.409 total time=  38.3s


/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

[CV 1/5] END activation=relu, batch_size=2, hidden_layer_sizes=(80, 16), max_iter=100, solver=adam;, score=-336.556 total time=  38.8s
[CV 4/5] END activation=relu, batch_size=2, hidden_layer_sizes=(80, 16), max_iter=100, solver=adam;, score=-171.226 total time=  38.9s


...

[CV 4/5] END activation=relu, batch_size=4, hidden_layer_sizes=(80, 10), max_iter=80, solver=lbfgs;, score=-147.025 total time=   0.5s
[CV 5/5] END activation=relu, batch_size=4, hidden_layer_sizes=(80, 10), max_iter=80, solver=lbfgs;, score=-116.228 total time=   0.5s


/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1607: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[CV 4/5] END activation=relu, batch_size=4, hidden_layer_sizes=(80, 10), max_iter=80, solver=adam;, score=-169.969 total time=  16.3s
[CV 5/5] END activation=relu, batch_size=4, hidden_layer_sizes=(80, 10), max_iter=80, solver=adam;, score=-143.183 total time=  16.1s


/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.



GridSearchCV(cv=5, estimator=MLPRegressor(random_state=123), n_jobs=-1,
             param_grid={'activation': ['relu'], 'batch_size': [2, 4],
                         'hidden_layer_sizes': [(80, 16), (100, 50), (80, 10)],
                         'max_iter': [100, 80], 'solver': ['adam', 'lbfgs']},
             scoring='neg_mean_absolute_error', verbose=5)

In [176]:

print('### Printing Best parameters ###')

mlp_regressor_grid2.best_params_

### Printing Best parameters ###


{'activation': 'relu',
 'batch_size': 4,
 'hidden_layer_sizes': (100, 50),
 'max_iter': 80,
 'solver': 'adam'}

In [177]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,r2_score

best_grid2 = mlp_regressor_grid2.best_estimator_
y_test_pred_mlpcv2 = best_grid2.predict(X_test)

mae_mlpcv2 = mean_absolute_error(y_test, y_test_pred_mlpcv2)
mse_mlpcv2 = mean_squared_error(y_test, y_test_pred_mlpcv2)
mape_mlpcv2 = mean_absolute_percentage_error(y_test, y_test_pred_mlpcv2)
r2_mlpcv2 = r2_score(y_test,y_test_pred_mlpcv2)

In [178]:
print("MAE_CV : ", mae_mlpcv2)
print("r2_CV : ", r2_mlpcv2)
print("mse_CV : ", mse_mlpcv2)
print("mape_CV: ", mape_mlpcv2)

MAE_CV :  156.2702260892766
r2_CV :  0.2539814787637834
mse_CV :  737458.1409852536
mape_CV:  878624030916699.6
